In [ ]:
import numpy as np
import pandas as pd
import re

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving clothes_explicit.csv to clothes_explicit (2).csv


In [ ]:
clothes_df = pd.read_csv('clothes_explicit.csv')
clothes_df.head()

,product_id,product_images,gender,product_category,product_name,details,category_type,material,product_subtype,weather_feature,coverage_feature,combined_features
0,0,[],women,ACCESSORIES,bar refill,creamy texture multifunction makeup bar availa...,Accessories,other,other,other,other,NaN
1,1,[],women,ACCESSORIES,lip oil,creamy lip oil glide like silk achieve uniform...,Accessories,silk,other,other,other,silk
2,2,[],women,ACCESSORIES,ultimatte matte liquid lipstick,slick favourite lipstick liquid version covera...,Accessories,other,other,other,other,NaN
3,3,[{'https://static.zara.net/photos///2022/I/2/1...,women,SHOES,face brush,small face brush application powder cream liqu...,Shoes,other,other,other,other,NaN
4,4,[],women,SHOES,face brush,design apply powder cream product perfect blus...,Shoes,other,other,other,other,NaN


# Preprocessing

In [ ]:
clothes_df['material'].unique()

array(['other', 'silk', 'satin', 'denim', 'faux leather', 'linen', 'knit',
       'cotton linen', 'organza', 'semisheer', 'rubberise',
       'linen viscose', 'poplin', 'cotton', 'knit wool',
       'cashmere knit wool', 'viscose', 'polyamide', 'nylon',
       'satin viscose', 'satiny', 'cotton poplin', 'faux suede',
       'chiffon', 'ramie', 'gabardine', 'gabardine gabardinestyle',
       'linen lyocell', 'lyocell', 'knit linen', 'semisheer viscose',
       'cotton knit', 'liocell viscose wool', 'leather', 'faux',
       'cotton jute linen', 'acetate jute', 'wool', 'knit viscose',
       'suede', 'faux patent', 'cashmere knit', 'alpaca wool',
       'alpaca knit wool', 'knit semisheer', 'knit poplin', 'denim knit',
       'organza satin semisheer', 'patent', 'polyurethane',
       'leather polyurethane', 'polyurethane rubberise', 'denim jute',
       'velvet', 'neoprene', 'leather suede',
       'denim polyurethane rubberise', 'gabardine gabardinetype',
       'cotton lyocell', 'cott

In [ ]:
clothes_df = clothes_df.fillna("").astype(str)

def describe_item(row):
    name = row.get("product_name", "").strip()
    material = row.get("material", "").strip() or "unspecified material"
    coverage = row.get("coverage_feature", "").strip() or "medium"
    return f"This is a {name} made of {material} with {coverage} coverage."

clothes_df["text_complex"] = clothes_df.apply(describe_item, axis=1)

In [ ]:
def apply_weather_rules(text):
    if any(word in text for word in ["wool", "fleece", "knit"]):
        text += " It is suitable for cold weather."

    elif any(word in text for word in ["linen", "crop", "sleeveless", "tank", "silk", "acetate jute", "viscose"]):
        text += " It is suitable for warm or hot weather."

    elif any(word in text for word in ["denim", "cotton", "long", "leather"]):
        text += " It is suitable for mild or cool weather."

    elif any(word in text for word in ["gabardine", "trench coat"]):
        text += " It is suitable for rainy weather."
    return text

clothes_df["text_complex"] = clothes_df["text_complex"].apply(apply_weather_rules)

In [ ]:
clothes_df.iloc[-3,-1]

'This is a tank slogan made of cotton with other coverage. It is suitable for warm or hot weather.'

In [ ]:
for cat in clothes_df["category_type"].unique():
    var_name = cat.replace(" ", "_").lower() + "_df"
    globals()[var_name] = clothes_df[clothes_df["category_type"] == cat].copy()

In [ ]:
accessories_df

,product_id,product_images,gender,product_category,product_name,details,category_type,material,product_subtype,weather_feature,coverage_feature,combined_features,text_complex
0,0,[],women,ACCESSORIES,bar refill,creamy texture multifunction makeup bar availa...,Accessories,other,other,other,other,,This is a bar refill made of other with other ...
1,1,[],women,ACCESSORIES,lip oil,creamy lip oil glide like silk achieve uniform...,Accessories,silk,other,other,other,silk,This is a lip oil made of silk with other cove...
2,2,[],women,ACCESSORIES,ultimatte matte liquid lipstick,slick favourite lipstick liquid version covera...,Accessories,other,other,other,other,,This is a ultimatte matte liquid lipstick made...
6,6,[{'https://static.zara.net/photos///2022/I/2/1...,women,ACCESSORIES,eyelash curler,allow eyelash curl raise single movement inten...,Accessories,other,other,other,other,,This is a eyelash curler made of other with ot...
8,8,[{'https://static.zara.net/photos///2022/I/2/1...,women,ACCESSORIES,eye brush,small eye brush design blend apply eye shadow ...,Accessories,other,other,other,other,,This is a eye brush made of other with other c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3822,4131,[{'https://static.zara.net/photos///2023/I/1/2...,men,ACCESSORIES,athleticz laken l oz thermos bottle,thermos bottle laken brand stainless steel cap...,Accessories,polyamide,bottle cap,insulate thermos,other,polyamide bottle cap insulate thermos,This is a athleticz laken l oz thermos bottle ...
3823,4132,[{'https://static.zara.net/photos///2023/I/0/2...,men,ACCESSORIES,baseball training cap,peak cap adjustable strap backzara athleticz,Accessories,other,cap,other,other,cap,This is a baseball training cap made of other ...
3827,4136,[{'https://static.zara.net/photos///2023/I/0/2...,men,ACCESSORIES,strappy cycling culotte,culotte highly stretchy fabric elastic strap f...,Accessories,other,other,other,other,,This is a strappy cycling culotte made of othe...
3828,4137,[{'https://static.zara.net/photos///2023/I/0/2...,men,ACCESSORIES,cap,cap stretchy breathable fabric secure fit inne...,Accessories,other,cap,other,other,cap,This is a cap made of other with other coverage.


In [ ]:
bottom_df.head()

,product_id,product_images,gender,product_category,product_name,details,category_type,material,product_subtype,weather_feature,coverage_feature,combined_features,text_complex
18,18,[],women,SKIRTS,z denim midi skirt,fade highwaist skirt feature fivepocket design...,Bottom,denim,denim skirt,other,midi,denim denim skirt midi,This is a z denim midi skirt made of denim wit...
19,19,[],women,SKIRTS,faux leather mini skirt chain,highwaist mini skirt lining metal chain link i...,Bottom,faux leather,skirt,other,mini,faux leather skirt mini,This is a faux leather mini skirt chain made o...
20,20,[],women,SKIRTS,skort pocket,midwaist skort adjustable elasticate waistband...,Bottom,other,skort,other,other,skort,This is a skort pocket made of other with othe...
21,21,[],women,SKIRTS,long satin skirt,long skirt high elastic waist,Bottom,satin,skirt,other,long,satin skirt long,This is a long satin skirt made of satin with ...
22,22,[],women,SKIRTS,knot midi skirt,highwaist flow midi skirt knot detail slit hem...,Bottom,other,skirt,other,midi,skirt midi,This is a knot midi skirt made of other with m...


In [ ]:
shoes_df.shape

(493, 13)

In [ ]:
top_df.shape

(1665, 13)

# Zero-Shot Classification

In [ ]:
#load the model
import torch
from transformers import pipeline
import tensorflow as tf

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


In [ ]:
#candidate labels
labels = ["cold weather",
          "cool weather",
          "mild weather",
          "warm weather",
          "hot weather",
          "rainy weather"]

### Classifying Accessories Category

In [ ]:
#zero-shot-classification on accessories category
# accessories_df = accessories_df.fillna("").astype(str)
# accessories_df_copy = accessories_df[accessories_df['combined_features'].str.strip() != ""].copy()
# accessories_df['text'] = accessories_df['combined_features'] + ' ' + accessories_df['product_name']
# texts = accessories_df['text'].tolist()

complex_texts = accessories_df["text_complex"].tolist()

batch_size = 16
results = []

for i in range(0, len(complex_texts), batch_size):
    batch_texts = complex_texts[i:i+batch_size]
    batch_results = classifier(batch_texts, candidate_labels=labels,
                               hypothesis_template="This outfit is designed to be worn in {} conditions.",
                               multi_label=True)

    for item in batch_results:
        results.append(item['labels'][0])

accessories_df['weather_label'] = results
accessories_df.head()

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


,product_id,product_images,gender,product_category,product_name,details,category_type,material,product_subtype,weather_feature,coverage_feature,combined_features,text_complex,weather_label
0,0,[],women,ACCESSORIES,bar refill,creamy texture multifunction makeup bar availa...,Accessories,other,other,other,other,,This is a bar refill made of other with other ...,cool weather
1,1,[],women,ACCESSORIES,lip oil,creamy lip oil glide like silk achieve uniform...,Accessories,silk,other,other,other,silk,This is a lip oil made of silk with other cove...,warm weather
2,2,[],women,ACCESSORIES,ultimatte matte liquid lipstick,slick favourite lipstick liquid version covera...,Accessories,other,other,other,other,,This is a ultimatte matte liquid lipstick made...,warm weather
6,6,[{'https://static.zara.net/photos///2022/I/2/1...,women,ACCESSORIES,eyelash curler,allow eyelash curl raise single movement inten...,Accessories,other,other,other,other,,This is a eyelash curler made of other with ot...,mild weather
8,8,[{'https://static.zara.net/photos///2022/I/2/1...,women,ACCESSORIES,eye brush,small eye brush design blend apply eye shadow ...,Accessories,other,other,other,other,,This is a eye brush made of other with other c...,mild weather


In [ ]:
accessories_df[accessories_df['weather_label'] == 'mild weather']

,product_id,product_images,gender,product_category,product_name,details,category_type,material,product_subtype,weather_feature,coverage_feature,combined_features,text_complex,weather_label
6,6,[{'https://static.zara.net/photos///2022/I/2/1...,women,ACCESSORIES,eyelash curler,allow eyelash curl raise single movement inten...,Accessories,other,other,other,other,,This is a eyelash curler made of other with ot...,mild weather
8,8,[{'https://static.zara.net/photos///2022/I/2/1...,women,ACCESSORIES,eye brush,small eye brush design blend apply eye shadow ...,Accessories,other,other,other,other,,This is a eye brush made of other with other c...,mild weather
9,9,[{'https://static.zara.net/photos///2023/V/2/1...,women,ACCESSORIES,makeup sponge,makeup sponge specially design liquid cream fo...,Accessories,other,other,water,other,water,This is a makeup sponge made of other with oth...,mild weather
1058,1168,[{'https://static.zara.net/photos///2023/I/0/1...,women,ACCESSORIES,chain link coin necklace,necklace feature metal link different size rai...,Accessories,other,necklace pendant,other,other,necklace pendant,This is a chain link coin necklace made of oth...,mild weather
1059,1169,[{'https://static.zara.net/photos///2023/I/0/1...,women,ACCESSORIES,leather belt square buckle,leather belt square buckle loop line material,Accessories,leather,belt,other,other,leather belt,This is a leather belt square buckle made of l...,mild weather
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3797,4105,[{'https://static.zara.net/photos///2023/I/1/2...,men,ACCESSORIES,rolltop sport backpack,multipocket backpack combination material exte...,Accessories,other,backpack,thermoseale,other,backpack thermoseale,This is a rolltop sport backpack made of other...,mild weather
3798,4106,[{'https://static.zara.net/photos///2023/I/0/2...,men,ACCESSORIES,pack training sock,pack pair sock spun cotton blend fabric rib tr...,Accessories,cotton,other,other,other,cotton,This is a pack training sock made of cotton wi...,mild weather
3804,4113,[{'https://static.zara.net/photos///2023/I/1/2...,men,ACCESSORIES,nylon belt bag,soft belt bag technical fabric upper main comp...,Accessories,nylon,bag belt,other,other,nylon bag belt,This is a nylon belt bag made of nylon with ot...,mild weather
3812,4121,[{'https://static.zara.net/photos///2023/I/1/2...,men,ACCESSORIES,sport backpack,multipocket backpack combination material colo...,Accessories,other,backpack,other,other,backpack,This is a sport backpack made of other with ot...,mild weather


In [ ]:
accessories_df.to_csv("new_classified_accessories.csv", index=False)
files.download("new_classified_accessories.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Classifying Shoes Category

In [ ]:
#zero-shot-classification on shoes category
# shoes_df = shoes_df.fillna("").astype(str)

# shoes_df['text'] = shoes_df['combined_features'] + ' ' + shoes_df['product_name']
# texts = shoes_df['text'].tolist()

complex_texts = shoes_df["text_complex"].tolist()

batch_size = 16
results = []

for i in range(0, len(complex_texts), batch_size):
    batch_texts = complex_texts[i:i+batch_size]
    batch_results = classifier(batch_texts, candidate_labels=labels,
                               hypothesis_template="This outfit is designed to be worn in {} conditions.",
                               multi_label=True)

    for item in batch_results:
        results.append(item['labels'][0])

shoes_df['weather_label'] = results
shoes_df.head()

,product_id,product_images,gender,product_category,product_name,details,category_type,material,product_subtype,weather_feature,coverage_feature,combined_features,text_complex,weather_label
3,3,[{'https://static.zara.net/photos///2022/I/2/1...,women,SHOES,face brush,small face brush application powder cream liqu...,Shoes,other,other,other,other,,This is a face brush made of other with other ...,mild weather
4,4,[],women,SHOES,face brush,design apply powder cream product perfect blus...,Shoes,other,other,other,other,,This is a face brush made of other with other ...,mild weather
7,7,[{'https://static.zara.net/photos///2022/I/2/1...,women,SHOES,eye brush,compact flat eye brush specially design apply ...,Shoes,other,other,other,other,,This is a eye brush made of other with other c...,mild weather
10,10,[],women,SHOES,face brush,round makeup brush ideal uniform application c...,Shoes,other,other,lightweight,other,lightweight,This is a face brush made of other with other ...,mild weather
11,11,[],women,SHOES,eyeliner,eyeliner ultrathin intensely pigment tip glide...,Shoes,other,other,other,other,,This is a eyeliner made of other with other co...,mild weather


In [ ]:
shoes_df[shoes_df['weather_label'] == 'warm weather']

,product_id,product_images,gender,product_category,product_name,details,category_type,material,product_subtype,weather_feature,coverage_feature,combined_features,text_complex,weather_label
1462,1579,[{'https://static.zara.net/photos///2023/I/1/1...,women,SHOES,satin ballet flat bow detail,satin ballet flat bow detail round toesole hei...,Shoes,satin,bow,other,other,satin bow,This is a satin ballet flat bow detail made of...,warm weather
1905,2090,[{'https://static.zara.net/photos///2023/I/1/1...,women,SHOES,split suede loafer buckle,split suede flat loafer metallic buckle detail...,Shoes,suede,loafer,other,other,suede loafer,This is a split suede loafer buckle made of su...,warm weather
1907,2092,[{'https://static.zara.net/photos///2023/I/1/1...,women,SHOES,technical running trainer,run trainers adjustable elastic drawstring pul...,Shoes,other,trainer,other,other,trainer,This is a technical running trainer made of ot...,warm weather
1908,2093,[{'https://static.zara.net/photos///2023/I/1/1...,women,SHOES,satin sandal rhinestone,satin highheel sandal thin strap rhinestone ro...,Shoes,satin,sandal,other,other,satin sandal,This is a satin sandal rhinestone made of sati...,warm weather
1917,2102,[{'https://static.zara.net/photos///2023/I/1/1...,women,SHOES,platform sneaker,trainer feature platform sole laceup fasten so...,Shoes,polyurethane,trainer,other,other,polyurethane trainer,This is a platform sneaker made of polyurethan...,warm weather
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3600,3908,[{'https://static.zara.net/photos///2023/I/1/2...,men,SHOES,chunky sandal,strappy sandal strap hookandloop strap fasten ...,Shoes,other,sandal,other,other,sandal,This is a chunky sandal made of other with oth...,warm weather
3785,4093,[{'https://static.zara.net/photos///2023/I/1/2...,men,SHOES,cross training sneaker,sneaker design training improved shock absorpt...,Shoes,other,other,absorption breathability,other,absorption breathability,This is a cross training sneaker made of other...,warm weather
3803,4112,[{'https://static.zara.net/photos///2023/I/1/2...,men,SHOES,run sneaker,run sneaker spume foam technology guarantee li...,Shoes,other,other,lightness,other,lightness,This is a run sneaker made of other with other...,warm weather
3809,4118,[{'https://static.zara.net/photos///2023/I/1/1...,men,SHOES,run sneaker,run sneaker spume foam technology guarantee li...,Shoes,other,other,lightness,other,lightness,This is a run sneaker made of other with other...,warm weather


In [ ]:
shoes_df.to_csv("new_classified_shoes.csv", index=False)
files.download("new_classified_shoes.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Classifying Tops Category

In [ ]:
#zero-shot-classification on top category
#top_df = top_df.fillna("").astype(str)

#top_df['text'] = top_df['product_name'] + ' ' + top_df['combined_features']
#texts = top_df['text'].tolist()

complex_texts = top_df["text_complex"].tolist()

batch_size = 16
results = []

for i in range(0, len(complex_texts), batch_size):
    batch_texts = complex_texts[i:i+batch_size]
    batch_results = classifier(batch_texts, candidate_labels=labels,
                               hypothesis_template="This item is typically worn in {}.",
                               multi_label=True)

    for item in batch_results:
        results.append(item['labels'][0])

top_df['weather_label'] = results
top_df.head()

,product_id,product_images,gender,product_category,product_name,details,category_type,material,product_subtype,weather_feature,coverage_feature,combined_features,text_complex,weather_label
5,5,[{'https://static.zara.net/photos///2022/V/2/1...,women,TOPS_BODYSUITS,kabuki brush,short smooth densebristle brush ideal apply fo...,Top,other,short,other,short,short short,This is a kabuki brush made of other with shor...,mild weather
12,12,[],women,JACKETS,coat nail varnish,quickdrye coat highlight colour nail ensure du...,Top,other,coat,other,other,coat,This is a coat nail varnish made of other with...,warm weather
13,13,[],women,TOPS_BODYSUITS,nail polish,superior wear naturally highperformance formul...,Top,other,other,other,long,long,This is a nail polish made of other with long ...,mild weather
14,14,[],women,TOPS_BODYSUITS,find,find,Top,other,other,other,other,,This is a find made of other with other coverage.,mild weather
16,16,[],women,TOPS_BODYSUITS,concealer,creamy luscious formula easy blend buildable m...,Top,satin,other,other,other,satin,This is a concealer made of satin with other c...,mild weather


In [ ]:
top_df[top_df['weather_label'] == 'warm weather']

,product_id,product_images,gender,product_category,product_name,details,category_type,material,product_subtype,weather_feature,coverage_feature,combined_features,text_complex,weather_label
12,12,[],women,JACKETS,coat nail varnish,quickdrye coat highlight colour nail ensure du...,Top,other,coat,other,other,coat,This is a coat nail varnish made of other with...,warm weather
851,961,[],women,TOPS_BODYSUITS,satin strapless,straight neckline expose shoulder feature matc...,Top,satin,other,other,other,satin,This is a satin strapless made of satin with o...,warm weather
853,963,[],women,TOPS_BODYSUITS,foil halter,vneck halter metallic fabric lining open invis...,Top,other,other,other,other,,This is a foil halter made of other with other...,warm weather
854,964,[],women,TOPS_BODYSUITS,asymmetric halterneck,pleated satin fabric halter neck feature open ...,Top,satin,halterneck,other,other,satin halterneck,This is a asymmetric halterneck made of satin ...,warm weather
855,965,[],women,TOPS_BODYSUITS,linen blend crop,short linen blend straight neckline expose sho...,Top,linen,short,other,crop short,linen short crop short,This is a linen blend crop made of linen with ...,warm weather
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816,4125,[{'https://static.zara.net/photos///2023/I/0/2...,men,T-SHIRTS,training tank,loosefitte tshirt lightweight technical stretc...,Top,other,tank tshirt,lightweight,sleeveless,tank tshirt lightweight sleeveless,This is a training tank made of other with sle...,warm weather
3818,4127,[{'https://static.zara.net/photos///2023/I/0/2...,men,T-SHIRTS,comfort workout tshirt,tshirt lightweight elastic fabric direction ro...,Top,other,short tshirt,lightweight,short,short tshirt lightweight short,This is a comfort workout tshirt made of other...,warm weather
3820,4129,[{'https://static.zara.net/photos///2023/I/0/2...,men,T-SHIRTS,rib tank,sleeveless wide round neckzara athleticz,Top,other,tank,other,sleeveless,tank sleeveless,This is a rib tank made of other with sleevele...,warm weather
3826,4135,[{'https://static.zara.net/photos///2023/I/0/2...,men,T-SHIRTS,texture compression tshirt,tshirt technical fabric high elasticity adapt ...,Top,other,tshirt,breathability,sleeveless,tshirt breathability sleeveless,This is a texture compression tshirt made of o...,warm weather


In [ ]:
top_df.to_csv("new_classified_top.csv", index=False)
files.download("new_classified_top.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Classifying Bottoms Category

In [ ]:
#zero-shot-classification on bottom category

# bottom_df = bottom_df.fillna("").astype(str)

# bottom_df['text'] = bottom_df['combined_features'] + ' ' + bottom_df['product_name']
# texts = bottom_df['text'].tolist()

complex_texts = bottom_df["text_complex"].tolist()

batch_size = 16
results = []

for i in range(0, len(complex_texts), batch_size):
    batch_texts = complex_texts[i:i+batch_size]
    batch_results = classifier(batch_texts, candidate_labels=labels,
                               hypothesis_template="This clothing is suitable for {} weather.",
                               multi_label=True)

    for item in batch_results:
        results.append(item['labels'][0])

bottom_df['weather_label'] = results
bottom_df.head()

,product_id,product_images,gender,product_category,product_name,details,category_type,material,product_subtype,weather_feature,coverage_feature,combined_features,text_complex,weather_label
18,18,[],women,SKIRTS,z denim midi skirt,fade highwaist skirt feature fivepocket design...,Bottom,denim,denim skirt,other,midi,denim denim skirt midi,This is a z denim midi skirt made of denim wit...,mild weather
19,19,[],women,SKIRTS,faux leather mini skirt chain,highwaist mini skirt lining metal chain link i...,Bottom,faux leather,skirt,other,mini,faux leather skirt mini,This is a faux leather mini skirt chain made o...,mild weather
20,20,[],women,SKIRTS,skort pocket,midwaist skort adjustable elasticate waistband...,Bottom,other,skort,other,other,skort,This is a skort pocket made of other with othe...,cold weather
21,21,[],women,SKIRTS,long satin skirt,long skirt high elastic waist,Bottom,satin,skirt,other,long,satin skirt long,This is a long satin skirt made of satin with ...,mild weather
22,22,[],women,SKIRTS,knot midi skirt,highwaist flow midi skirt knot detail slit hem...,Bottom,other,skirt,other,midi,skirt midi,This is a knot midi skirt made of other with m...,cool weather


In [ ]:
bottom_df[bottom_df['weather_label'] == 'warm weather']

,product_id,product_images,gender,product_category,product_name,details,category_type,material,product_subtype,weather_feature,coverage_feature,combined_features,text_complex,weather_label
23,23,[],women,SKIRTS,linen blend crossover skirt,midi skirt linen blend feature asymmetric wais...,Bottom,linen,belt skirt,other,midi,linen belt skirt midi,This is a linen blend crossover skirt made of ...,warm weather
29,29,[],women,SKIRTS,rustic texture skirt,midrise midi skirt cotton linen blend adjustab...,Bottom,cotton linen,skirt,other,midi,cotton linen skirt midi,This is a rustic texture skirt made of cotton ...,warm weather
30,30,[],women,SKIRTS,satin pleated skirt,highwaist midi skirt invisible zip fastening,Bottom,satin,skirt,other,midi,satin skirt midi,This is a satin pleated skirt made of satin wi...,warm weather
40,40,[],women,SKIRTS,voluminous midi skirt,highwaist midi skirt elastic waist pocket plea...,Bottom,other,skirt,other,midi,skirt midi,This is a voluminous midi skirt made of other ...,warm weather
47,47,[],women,SKIRTS,long linen skirt,long midrise skirt linen feature adjustable dr...,Bottom,linen,skirt,other,long,linen skirt long,This is a long linen skirt made of linen with ...,warm weather
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252,3560,[{'https://static.zara.net/photos///2023/I/0/2...,men,TROUSERS,linen suit trouser,regular fit trouser waist pleat pocket rear we...,Bottom,linen,suit trouser,other,other,linen suit trouser,This is a linen suit trouser made of linen wit...,warm weather
3253,3561,[{'https://static.zara.net/photos///2023/I/0/2...,men,TROUSERS,linen suit trouser,regularfit trouser pocket welt pocket zip fly ...,Bottom,linen,suit trouser,other,other,linen suit trouser,This is a linen suit trouser made of linen wit...,warm weather
3254,3562,[{'https://static.zara.net/photos///2023/I/0/2...,men,TROUSERS,stripe linenviscose trouser,relaxedfit trouser adjustable elasticate waist...,Bottom,linenviscose,belt trouser,other,other,linenviscose belt trouser,This is a stripe linenviscose trouser made of ...,warm weather
3257,3565,[{'https://static.zara.net/photos///2023/V/0/1...,men,SHORTS,linen bermuda short,bermuda short linen fabric adjustable elastica...,Bottom,linen,bermuda short,other,short,linen bermuda short short,This is a linen bermuda short made of linen wi...,warm weather


In [ ]:
bottom_df.to_csv("new_classified_bottom.csv", index=False)
files.download("new_classified_bottom.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Recommendation

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[ 6.76568970e-02  6.34958893e-02  4.87131067e-02  7.93049634e-02
   3.74480449e-02  2.65277340e-03  3.93749550e-02 -7.09847454e-03
   5.93613833e-02  3.15369889e-02  6.00981116e-02 -5.29051647e-02
   4.06067669e-02 -2.59308256e-02  2.98428144e-02  1.12693582e-03
   7.35148862e-02 -5.03819585e-02 -1.22386619e-01  2.37028375e-02
   2.97265332e-02  4.24768403e-02  2.56337766e-02  1.99517491e-03
  -5.69190979e-02 -2.71598268e-02 -3.29035521e-02  6.60248324e-02
   1.19007103e-01 -4.58791293e-02 -7.26214647e-02 -3.25840004e-02
   5.23413382e-02  4.50552590e-02  8.25299695e-03  3.67024392e-02
  -1.39415637e-02  6.53918907e-02 -2.64272504e-02  2.06358891e-04
  -1.36642987e-02 -3.62810269e-02 -1.95043534e-02 -2.89738569e-02
   3.94270420e-02 -8.84091035e-02  2.62426748e-03  1.36714121e-02
   4.83063087e-02 -3.11565865e-02 -1.17329188e-01 -5.11690304e-02
  -8.85287896e-02 -2.18962505e-02  1.42986495e-02  4.44167927e-02
  -1.34815276e-02  7.43392929e-02  2.66382527e-02 -1.98762529e-02
   1.79191

In [ ]:
temperature_mapping = {
            (5, 10): "5-10°C cold weather",
            (10, 15): "10-15°C cool weather",
            (15, 20): "15-20°C mild weather",
            (20, 25): "20-25°C warm weather",
            (25, 30): "25-30°C hot weather"
        }

In [ ]:
def temperature_to_label(temperature):
        """
        Convert numeric temperature to label
        """
        try:
            temp = float(temperature)
            for (low, high), label in temperature_mapping.items():
                if low <= temp <= high:
                    return label
            if temp < 5:
                return "5-10°C cold weather"
            elif temp > 30:
                return "25-30°C hot weather"
        except ValueError:
            raise ValueError(f"Invalid temperature: {temperature}. Please enter 5-30.")

In [ ]:
temperature = '25'
temperature_label = temperature_to_label(temperature)
temp_range = temperature_label.split(' ')[0]

In [ ]:
#load MoritzLaurer model
import torch
from transformers import pipeline
import tensorflow as tf

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [ ]:
# RANDOM SAMPLE - This is the key speed improvement!
sample_size = 5

#Sample from TOP category

if len(top_df) > sample_size:
    sample_df = top_df.sample(sample_size) #, random_state=42)
    print(f"🎯 Sampling {sample_size} random items from {len(top_df)} total Top items")
else:
    sample_df = top_df
    print(f"🎯 Using all {len(sample_df)} top items")



# Prepare texts from sampled items
texts = []
for idx, row in sample_df.iterrows():
    text = f"{row['product_name']}.{row['details']}"
    texts.append(text)



# Single batch classification (fast!)
print("⚡ Classifying sampled items in one batch...")
results = classifier(texts,
               candidate_labels=[temperature_label],
               hypothesis_template="This item is comfortable to wear when the weather is {}.",
               multi_label=False
              )


# Handle results
if isinstance(results, dict):
    scores = [results['scores'][0]]
else:
    scores = [result['scores'][0] for result in results]


# Get top K indices from sampled items
scores = np.array(scores)
top_indices = np.argsort(scores)[::-1][:5]
scores, top_indices


# Build results
top_recommendations = []
for i, idx in enumerate(top_indices, 1):
    item = sample_df.iloc[idx]
    top_recommendations.append({'rank': i,
                            'product_name': item['product_name'],
                            'details': item['details'],
                            'confidence': round(scores[idx], 3),
                            'temperature': f"{temperature}°C",
                            'temperature_range': temp_range,
                            'sampled_from_total': f"{len(sample_df)}/{len(clothes_df)} items"})
top_recommendations[0]['product_name']

🎯 Sampling 5 random items from 1620 total Top items
⚡ Classifying sampled items in one batch...


'faux leather bomber jacket'

In [ ]:
# RANDOM SAMPLE - This is the key speed improvement!
sample_size = 5

#Sample from BOTTOM category

if len(bottom_df) > sample_size:
    sample_df = bottom_df.sample(sample_size) #, random_state=42)
    print(f"🎯 Sampling {sample_size} random items from {len(bottom_df)} total bottom items")
else:
    sample_df = bottom_df
    print(f"🎯 Using all {len(sample_df)} bottom items")



# Prepare texts from sampled items
texts = []
for idx, row in sample_df.iterrows():
    text = f"{row['product_name']}.{row['details']}"
    texts.append(text)



# Single batch classification (fast!)
print("⚡ Classifying sampled items in one batch...")
results = classifier(texts,
               candidate_labels=[temperature_label],
               hypothesis_template="This item is comfortable to wear when the weather is {}.",
               multi_label=False
              )


# Handle results
if isinstance(results, dict):
    scores = [results['scores'][0]]
else:
    scores = [result['scores'][0] for result in results]


# Get top K indices from sampled items
scores = np.array(scores)
top_indices = np.argsort(scores)[::-1][:5]
scores, top_indices


# Build results
bottom_recommendations = []
for i, idx in enumerate(top_indices, 1):
    item = sample_df.iloc[idx]
    bottom_recommendations.append({'rank': i,
                            'product_name': item['product_name'],
                            'details': item['details'],
                            'confidence': round(scores[idx], 3),
                            'temperature': f"{temperature}°C",
                            'temperature_range': temp_range,
                            'sampled_from_total': f"{len(sample_df)}/{len(clothes_df)} items"})
bottom_recommendations[0]['product_name']

🎯 Sampling 5 random items from 599 total bottom items
⚡ Classifying sampled items in one batch...


'straight fit trouser button'

In [ ]:
# RANDOM SAMPLE - This is the key speed improvement!
sample_size = 5

#Sample from SHOES category

if len(shoes_df) > sample_size:
    sample_df = shoes_df.sample(sample_size) #, random_state=42)
    print(f"🎯 Sampling {sample_size} random items from {len(shoes_df)} total shoes items")
else:
    sample_df = shoes_df
    print(f"🎯 Using all {len(sample_df)} shoes items")



# Prepare texts from sampled items
texts = []
for idx, row in sample_df.iterrows():
    text = f"{row['product_name']}.{row['details']}"
    texts.append(text)



# Single batch classification (fast!)
print("⚡ Classifying sampled items in one batch...")
results = classifier(texts,
               candidate_labels=[temperature_label],
               hypothesis_template="This item is comfortable to wear when the weather is {}.",
               multi_label=False
              )


# Handle results
if isinstance(results, dict):
    scores = [results['scores'][0]]
else:
    scores = [result['scores'][0] for result in results]


# Get top K indices from sampled items
scores = np.array(scores)
top_indices = np.argsort(scores)[::-1][:5]
scores, top_indices


# Build results
shoes_recommendations = []
for i, idx in enumerate(top_indices, 1):
    item = sample_df.iloc[idx]
    shoes_recommendations.append({'rank': i,
                            'product_name': item['product_name'],
                            'details': item['details'],
                            'confidence': round(scores[idx], 3),
                            'temperature': f"{temperature}°C",
                            'temperature_range': temp_range,
                            'sampled_from_total': f"{len(sample_df)}/{len(clothes_df)} items"})
shoes_recommendations[0]['product_name']

🎯 Sampling 5 random items from 479 total shoes items
⚡ Classifying sampled items in one batch...


'multipiece hightop sneaker'

In [ ]:
# RANDOM SAMPLE - This is the key speed improvement!
sample_size = 5

#Sample from ACCESSORIES category

if len(accessories_df) > sample_size:
    sample_df = accessories_df.sample(sample_size) #, random_state=42)
    print(f"🎯 Sampling {sample_size} random items from {len(accessories_df)} total accessories items")
else:
    sample_df = accessories_df
    print(f"🎯 Using all {len(sample_df)} accessories items")



# Prepare texts from sampled items
texts = []
for idx, row in sample_df.iterrows():
    text = f"{row['product_name']}.{row['details']}"
    texts.append(text)



# Single batch classification (fast!)
print("⚡ Classifying sampled items in one batch...")
results = classifier(texts,
               candidate_labels=[temperature_label],
               hypothesis_template="This item is comfortable to wear when the weather is {}.",
               multi_label=False
              )


# Handle results
if isinstance(results, dict):
    scores = [results['scores'][0]]
else:
    scores = [result['scores'][0] for result in results]


# Get top K indices from sampled items
scores = np.array(scores)
top_indices = np.argsort(scores)[::-1][:5]
scores, top_indices


# Build results
accessories_recommendations = []
for i, idx in enumerate(top_indices, 1):
    item = sample_df.iloc[idx]
    accessories_recommendations.append({'rank': i,
                            'product_name': item['product_name'],
                            'details': item['details'],
                            'confidence': round(scores[idx], 3),
                            'temperature': f"{temperature}°C",
                            'temperature_range': temp_range,
                            'sampled_from_total': f"{len(sample_df)}/{len(clothes_df)} items"})
accessories_recommendations[0]['product_name']

🎯 Sampling 5 random items from 181 total accessories items
⚡ Classifying sampled items in one batch...


'pack noshow sock'

In [ ]:
labels

['5-10°C cold weather',
 '10-15°C cool weather',
 '15-20°C mild weather',
 '20-25°C warm weather',
 '25-30°C hot weather',
 'rain']

In [ ]:
# Save to a JSON file
import json

config = {
    "model": "facebook/bart-large-mnli",
    "labels": ['5-10°C cold weather',
               '10-15°C cool weather',
               '15-20°C mild weather',
               '20-25°C warm weather',
               '25-30°C hot weather',
               'rain'],
    "hypothesis_template": "This item is comfortable to wear when the weather is {}."
}

with open("zero_shot_config.json", "w") as f:
    json.dump(config, f, indent=4)

In [ ]:
files.download("zero_shot_config.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>